Дополним датасет лемматизированным текстом комментариев. Для этого сохраним леммы частых токенов и потом обработаем все тексты.

In [12]:
import pandas as pd
from string import punctuation
import gensim
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
tqdm.pandas()
from string import punctuation
from collections import Counter
import ufal.udpipe
from model import Model
import conllu
from collections import defaultdict
from collections import Counter
from nltk.parse import DependencyGraph
import networkx as nx
import matplotlib.pyplot as plt
from nltk.text import Text
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
import plotly.express as px
import emoji

morph = MorphAnalyzer()

In [10]:
data = pd.read_csv('final_df.csv')

In [ ]:
lemmes = {}
for key in tqdm(dict(Counter(tokens).most_common(100000))):
    lemmes[key] = morph.parse(key)[0].normal_form

    
def lemmed_sent(sent):
    new_sent = []
    for p in punctuation:
        sent = sent.replace(p, '')
    for word in sent.split():
        razbor = ''
        if word in lemmes.keys():
            razbor = lemmes[word]
        else:
            razbor = morph.parse(word)[0].normal_form
        new_sent.append(razbor)
    return ' '.join(new_sent)

data['lemmed_comment'] = data['text'].progress_apply(lemmed_sent)

In [ ]:
data.to_csv('new_final_df.csv')

Соберём список интенсификаторов. 

In [ ]:
ud_model = Model('russian-taiga-ud-2.4-190531.udpipe')

In [ ]:
reviews = '\n'.join(str(a) for a in list(data['text']))
with open('comms.txt', 'w', encoding='utf-8') as f:
    f.write(reviews)
    
data = gensim.models.word2vec.LineSentence('comms.txt')
%time model_rev = gensim.models.Word2Vec(data, vector_size=500, window=5, min_count=5, epochs=10)

In [ ]:
def get_more_words(list_start):
    final_namings = []
    final_namings.extend(list_start)
    for name in list_start:
        for (word, cos) in model_rev.wv.most_similar(name):
            if cos >= 0.5:
                if word[-1] in punctuation:
                    final_namings.append(word[:-1])
                else:
                    final_namings.append(word)
    final_namings = [a.lower() for a in final_namings]
    final_namings = list(set(final_namings))
    return final_namings

In [ ]:
my_namings = ['очень', 'крайне']
final = get_more_words(my_namings)

final_2 = get_more_words(final)

final_3 = get_more_words(final_2)

real_final = ['абсолютно', 'адски', 'безгранично', 'безмерно', 'безумно', 
              'бесконечно', 'беспредельно', 'более-менее', 'весьма', 'всецело', 
              'давольно', 'дико', 'действительно', 'довольно', 'достаточно', 
              'дюже', 'едва', 'еле', 'исключительно', 'катастрофически', 
              'колоссально', 'крайне',  'критически', 'максимально', 'наиболее', 
              'наименее', 'настолько', 'невероятно', 'недостаточно', 'невыразимо', 
              'неимоверно', 'немного', 'непередаваемо', 'несказанно', 'несколько', 
              'отвратительно', 'отчасти', 'оч', 'очень', 'очень-очень', 'оочень', 
              'ооочень', 'ооооочень', 'полностью', 'поразительно', 'почти', 
              'практически', 'реально', 'сильно', 'сказочно', 'слегка', 'слишком', 
              'смертельно', 'совершенно', 'совсем', 'столь', 'страшно', 'удивительно', 
              'ужасно', 'чертовски', 'чрезвычайно', 'чудовищно', 'чуть-чуть',  
              'фантастически', 'феноменально']

Подготовим синтаксический разбор и функцию, которая будет находить что-то, похожее на наречные интенсификаторы.

In [ ]:
def get_conllu(model, te):
    sentences = model.tokenize(te)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu_text = model.write(sentences, "conllu")
    trees = []
    for sent in conllu_text.split('\n\n'):
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))
    return trees

In [ ]:
def find_adv_intensifiers(t):
    intense_cols = []
    trees = get_conllu(ud_model, t)
    for tr in trees:
        if tr.endswith('\s'):
            tr = tr[:-3]
        new_tr = tr.replace('SpacesAfter= ', 'SpacesAfter=')
        new_tr = new_tr.replace('SpacesAfter=\t', 'SpacesAfter=')
        new_tr = new_tr.replace('SpaceAfter= ', 'SpaceAfter=')
        new_tr = new_tr.replace('SpaceAfter=\t', 'SpaceAfter=')
        try:
            d = DependencyGraph(new_tr)
            d.root = d.nodes[0]
            trips = list(d.triples())
            for triple in trips:
                if triple[2][0].lower() in real_final and triple[1] in ['advmod', 'obl']:
                    intense_cols.append(triple[2][0] + ' ' + triple[0][0])
                
        except:
            pass
    return intense_cols

In [ ]:
data['intensified'] = data['text'].progress_apply(find_adv_intensifiers)

In [ ]:
data[data['intensified'] != "[]"]

In [ ]:
int_data = intense_data['intensified'].to_list()
new_int_data = []
for a in fake_data:
    a = a.replace('[', '')
    a = a.replace(']', '')
    a = a.replace("'", '')
    a = a.replace("\"", '')
    a = a.split(', ')
    new_int_data.extend(a)

In [ ]:
intense_data.loc[intense_data['intensified']== '[]', 'intensified'] = None

In [ ]:
intensifiers_table = []
for comment in parsed:
    if len(comment['intensified']) > 2:
        a = comment['intensified'].replace('[', '')
        a = a.replace('[', '')
        a = a.replace(']', '')
        a = a.replace("'", '')
        a = a.replace("\"", '')
        ints = a.split(', ')
        if len(ints) == 1:
            one = {}
            one['intensified'] = ints[0]
            one['text'] = comment['text']
            one['from_id'] = comment['from_id'] 
            one['group'] = comment['group']
            one['datetime'] = comment['datetime']
            one['name'] = comment['name']
            one['bdate'] = comment['bdate']
            one['city'] = comment['city']
            one['sex'] = comment['sex']
            intensifiers_table.append(one)
        else:
            for i in ints:
                one = {}
                one['intensified'] = i
                one['text'] = comment['text']
                one['from_id'] = comment['from_id'] 
                one['group'] = comment['group']
                one['datetime'] = comment['datetime']
                one['name'] = comment['name']
                one['bdate'] = comment['bdate']
                one['city'] = comment['city']
                one['sex'] = comment['sex']
                intensifiers_table.append(one)

In [ ]:
intense_df = pd.DataFrame(intensifiers_table)

In [ ]:
def calculate_pmi(intensifier, word, freq, df):
    p_very = len(df)/len(lemmed_corp)
    p_w = sum([1 for a in lemmed_corp if a.lower() == word])/len(lemmed_corp)
    p_very_w = freq/len(lemmed_corp)
    if p_w*p_very > 0:
        spl = p_very_w/(p_w*p_very)
        return math.log2(spl)
    return 0

In [ ]:
def change_pos(all_pos):
    values = values = ['PRCL', 'NUMR']
    all_pos = [v for v in all_pos if v not in values]
    all_pos = ['глагол' if x in ['VERB', 'INFN', 'GRND', 'PRTF', 'PRTS'] else x for x in all_pos]
    all_pos = ['наречие' if x in ['ADVB'] else x for x in all_pos]
    all_pos = ['прилагательное' if x in ['ADJF', 'ADJS', 'COMP'] else x for x in all_pos]
    all_pos = ['существительное' if x in ['NOUN'] else x for x in all_pos]
    all_pos = ['предикат' if x in ['PRED'] else x for x in all_pos]
    all_pos = ['междометие' if x in ['INTJ'] else x for x in all_pos]
    all_pos = ['местоимение' if x in ['NPRO'] else x for x in all_pos]
    all_pos = ['эмоджи' if x in ['EMOJ'] else x for x in all_pos]
    p = dict(Counter(all_pos))
    return '\n'.join(': '.join(str(e) for e in a) for a in list({k: v for k, v in sorted(p.items(), key=lambda item: item[1])}.items())[::-1])

In [ ]:
def all_word_stat(word):
    print(f'Статистика для слова {word}')
    word_df = intense_df[intense_df['intensifier'] == word]
    lentach_percent = round(len(word_df[word_df['group'] == 'Лентач'])/len(word_df)*100, 2)
    palata_percent = round(len(word_df[word_df['group'] == 'Палата №6'])/len(word_df)*100, 2)
    ria_percent = round(len(word_df[word_df['group'] == 'РИАНовости'])/len(word_df)*100, 2)
    print(f'Соотношение в группах Лентач/РиаНовости/Палата №6: {lentach_percent}/{palata_percent}/{ria_percent}')
    ipm_fem = len(word_df[word_df['sex'] == 1])/female_total_words*1000000
    ipm_male = len(word_df[word_df['sex'] == 2])/male_total_words*1000000
    print(f'IPM для слова {word} для женщин/мужчин: {round(ipm_fem, 2)}/{round(ipm_male, 2)}')
    if ipm_male > 0 and ipm_fem > 0:
        if ipm_fem > ipm_male:
            print(f'Женщины употребляют слово {word} чаще в {round(ipm_fem/ipm_male, 2)} раз(а)')
        else:
            print(f'Мужчины употребляют слово {word} чаще в {round(ipm_male/ipm_fem, 2)} раз(а)')
    print(f'Распределение по частям речи: {change_pos(word_df["pos"].to_list())}')
    vv = dict(Counter(word_df['normed'].to_list()))
    pmis = {}
    vv = {k: v for k, v in vv.items() if v > 1}
    print(vv)
    for second in vv.keys():
        print(f'PMI для слов {word} + {second}')
        pmis[second] = round(calculate_pmi(word, second, len(word_df[word_df['normed'] == second]), word_df), 3)
    print(list({k: v for k, v in sorted(pmis.items(), key=lambda item: item[1])}.items())[::-1][:10])
    print(sorted(dict(Counter([a for a in word_df['age_group'] if a != None])).items()))
    print('_______________________')

In [ ]:
def get_year(st):
    if len(st.split('.')) == 3:
        year = st.split('.')[2]
        if int(year) > 1940:
            return year
    return 0

def get_age(birth, dt):
    return int(dt.split()[0].split('.')[2]) - int(birth)

def age_group(age):
    if 14 <= age <= 24:
        return '14-24'
    elif 25 <= age <= 30:
        return '25-30'
    elif 31 <= age <= 36:
        return '31-36'
    elif 37 <= age <= 45:
        return '37-45'
    elif 46 <= age <= 90:
        return '46+'

intense_df['birthyear'] = intense_df['bdate'].apply(get_year)

intense_df['age'] = intense_df.apply(lambda x: get_age(x.birthyear, x.datetime), axis=1)

intense_df['age_group'] = intense_df['age'].apply(age_group)